In [81]:
import tensorflow as tf
import numpy as np


class Actor(tf.keras.Model):
    def __init__(self):
        super(Actor,self).__init__()
        self.lstm = tf.keras.layers.LSTM(5, return_sequences=True, stateful=True,input_shape=(1,1))#, input_shape=(None,None,1))
        
    def call(self, inputs):
        feat= self.lstm(inputs)
        return feat
    
    def reset_states_workaround(self, new_batch_size=1):
        self.lstm.states = [tf.Variable(tf.random.uniform((new_batch_size,5))), tf.Variable(tf.random.uniform((new_batch_size,5)))]
        self.lstm.input_spec = [tf.keras.layers.InputSpec(shape=(new_batch_size,None,1), ndim=3)]



In [87]:
actor = Actor()
context1 = tf.reshape(np.array([0.]*10),(10,1,1))
actor(context1)

new_batch_size=1
context1 = tf.reshape(np.array([0.]*new_batch_size),(new_batch_size,1,1))
actor.reset_states_workaround(new_batch_size=new_batch_size)
actor(context1)





If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



<tf.Tensor: shape=(1, 1, 5), dtype=float32, numpy=
array([[[ 0.2599167 ,  0.20581838,  0.16166003, -0.02506167,
          0.253754  ]]], dtype=float32)>

In [91]:
actor = Actor()
@tf.function
def g(actor):
    context1 = tf.reshape(np.array([0.]*10),(10,1,1))
    preds = actor(context1)
    return preds

g(actor)    
@tf.function
def g2(actor):
    actor.reset_states_workaround(new_batch_size=1)
    
    context1 = tf.reshape(np.array([0.]*1),(1,1,1))
    preds = actor(context1)
    return preds

g2(actor)    


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



ValueError: in user code:

    <ipython-input-91-96d5ee243d14>:11 g2  *
        actor.reset_states_workaround(new_batch_size=1)
    <ipython-input-81-c6d0222f0cdb>:15 reset_states_workaround  *
        self.lstm.states = [tf.Variable(tf.random.uniform((new_batch_size,5))), tf.Variable(tf.random.uniform((new_batch_size,5)))]
    /home/cooper-cooper/.local/lib/python3.6/site-packages/tensorflow/python/ops/variables.py:261 __call__  **
        return cls._variable_v2_call(*args, **kwargs)
    /home/cooper-cooper/.local/lib/python3.6/site-packages/tensorflow/python/ops/variables.py:255 _variable_v2_call
        shape=shape)
    /home/cooper-cooper/.local/lib/python3.6/site-packages/tensorflow/python/ops/variables.py:66 getter
        return captured_getter(captured_previous, **kwargs)
    /home/cooper-cooper/.local/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py:511 invalid_creator_scope
        "tf.function-decorated function tried to create "

    ValueError: tf.funct

In [8]:
actor.lstm.input_spec = [tf.keras.layers.InputSpec(shape=(20, None, 1), ndim=3)]

In [9]:
actor.lstm.reset_states([tf.ones((20,5)), tf.ones((20,5))])

ValueError: State 0 is incompatible with layer lstm: expected shape=(20, <tf.Variable 'actor/lstm/Variable:0' shape=(10, 5) dtype=float32, numpy=
array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]], dtype=float32)>), found shape=(20, 5)

In [13]:
actor.lstm.states

[<tf.Variable 'actor/lstm/Variable:0' shape=(10, 5) dtype=float32, numpy=
 array([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'actor/lstm/Variable:0' shape=(10, 5) dtype=float32, numpy=
 array([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]], dtype=float32)>]

In [14]:
actor = Actor()

In [15]:
actor.lstm.states

[None, None]

In [16]:
actor.lstm.input_spec

[InputSpec(ndim=3)]

In [18]:
actor.lstm.stateful

True

<tf.Tensor: shape=(10, 1, 5), dtype=float32, numpy=
array([[[ 3.2651640e-02,  1.9661312e-04,  2.7104491e-02, -6.1883725e-02,
         -6.2910609e-02]],

       [[ 2.1093087e-02,  1.1662899e-04,  1.7458558e-02, -4.2562421e-02,
         -4.3213338e-02]],

       [[ 1.2611021e-02,  6.5578432e-05,  1.0420575e-02, -2.6453516e-02,
         -2.6859863e-02]],

       [[ 4.7846124e-02,  3.2356600e-04,  3.9920077e-02, -8.1996761e-02,
         -8.3752029e-02]],

       [[ 4.5526605e-02,  3.0236636e-04,  3.7951563e-02, -7.9341203e-02,
         -8.0960602e-02]],

       [[ 1.5011438e-02,  7.9421603e-05,  1.2409455e-02, -3.1163596e-02,
         -3.1639338e-02]],

       [[ 2.4854733e-02,  1.4126317e-04,  2.0589873e-02, -4.9202740e-02,
         -4.9966544e-02]],

       [[ 2.8048186e-02,  1.6320977e-04,  2.3253912e-02, -5.4578271e-02,
         -5.5443905e-02]],

       [[ 2.0252764e-02,  1.1129928e-04,  1.6759967e-02, -4.1035052e-02,
         -4.1661453e-02]],

       [[ 3.5154946e-02,  2.1569952e-04

In [23]:
actor.lstm.states

[None, None]

In [24]:
actor.lstm.input_spec

[InputSpec(shape=(None, None, 1), ndim=3)]

In [25]:
actor.stateful

False

In [62]:
actor = Actor()
actor.lstm.stateful = False
print(actor.lstm.input_spec)
actor(tf.random.uniform((10,1,1)))
print(actor.lstm.input_spec)
actor.lstm.stateful=True
actor.lstm.states = [tf.Variable(tf.random.uniform((10,5))), tf.Variable(tf.random.uniform((10,5)))]
actor(tf.random.uniform((10,1,1)))
print(actor.lstm.input_spec)
print()

[InputSpec(ndim=3)]
[InputSpec(shape=(None, None, 1), ndim=3)]
[InputSpec(shape=(None, None, 1), ndim=3)]



********

In [63]:
actor = Actor()
actor.lstm.stateful = False
print(actor.lstm.input_spec)
actor(tf.random.uniform((20,1,1)))
print(actor.lstm.input_spec)
actor.lstm.stateful=True
actor.lstm.states = [tf.Variable(tf.random.uniform((10,5))), tf.Variable(tf.random.uniform((10,5)))]
actor(tf.random.uniform((10,1,1)))
print(actor.lstm.input_spec)
print()

[InputSpec(ndim=3)]
[InputSpec(shape=(None, None, 1), ndim=3)]
[InputSpec(shape=(None, None, 1), ndim=3)]



In [79]:
actor = Actor()

In [80]:

@tf.function
def g(actor):
    context1 = tf.reshape(np.array([0.]*10),(10,1,1))
    preds = actor(context1)
    return preds

g(actor)    


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



<tf.Tensor: shape=(10, 1, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.]]], dtype=float32)>

In [74]:
context1 = tf.reshape(np.array([0.]*1),(1,1,1))
preds = actor(context1)

In [75]:
actor.lstm.input_spec

[InputSpec(shape=(10, None, 1), ndim=3)]

In [77]:

@tf.function
def g(actor):
    context1 = tf.reshape(np.array([0.]*10),(10,1,1))
    preds = actor(context1)
    return preds

g(actor)    


@tf.function
def g2(actor):
    context1 = tf.reshape(np.array([0.]*1),(1,1,1))
    preds = actor(context1)
    return preds

g2(actor)    

<tf.Tensor: shape=(1, 1, 5), dtype=float32, numpy=
array([[[-0.14252928,  0.38017863,  0.15557224,  0.21203084,
          0.2901796 ]]], dtype=float32)>